# Практическое задание №7 по теме "Сверточные нейронные сети для анализа текста".

In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Загрузка библиотек:

In [2]:
import pandas as pd
import numpy as np
from string import punctuation
from pymorphy2 import MorphAnalyzer
import re

from gensim.models import KeyedVectors
import gensim
from tensorflow.keras.layers import Embedding

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
#from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

### Загрузка данных:

In [4]:
df_train = pd.read_csv("/content/drive/MyDrive/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/test.csv")
df_val = pd.read_csv("/content/drive/MyDrive/val.csv")

In [5]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


### Препроцессинг данных:

In [6]:
sw = set(stopwords.words('russian'))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [7]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

tokens = word_tokenize(train_corpus)

tokens_filtered = [word for word in tokens if word.isalnum()]

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [8]:
tokens_filtered_top[:10]

['rt',
 'это',
 'хотеть',
 'd',
 'день',
 'сегодня',
 'такой',
 'быть',
 'очень',
 'мой']

In [9]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [10]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

### Разделение данных:

In [11]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)

### Модель с Embedding по умолчанию:

Создание модели:

In [13]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Обучение модели:

In [15]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 60s 186ms/step - loss: 0.6252 - accuracy: 0.6187 - val_loss: 0.6166 - val_accuracy: 0.6287
Epoch 2/20
319/319 [==============================] - 60s 189ms/step - loss: 0.6125 - accuracy: 0.6341 - val_loss: 0.6143 - val_accuracy: 0.6299
Epoch 3/20
319/319 [==============================] - 65s 203ms/step - loss: 0.6070 - accuracy: 0.6381 - val_loss: 0.6135 - val_accuracy: 0.6302
Epoch 4/20
319/319 [==============================] - 60s 189ms/step - loss: 0.6018 - accuracy: 0.6446 - val_loss: 0.6136 - val_accuracy: 0.6323


In [16]:
score1 = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)

45/45 [==============================] - 2s 53ms/step - loss: 0.6175 - accuracy: 0.6253


### Модель с Embedding c предобученными векторами:

Загружаем вектора:

In [17]:
wv_from_text = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/ruscorpora_upos_skipgram_300_5_2018.vec', binary=False)

In [18]:
def gensim_to_keras_embedding(keyed_vectors, train_embeddings=False):
    """Get a Keras 'Embedding' layer with weights set from Word2Vec model's learned word embeddings.

    Parameters
    ----------
    train_embeddings : bool
        If False, the returned weights are frozen and stopped from being updated.
        If True, the weights can / will be further updated in Keras.

    Returns
    -------
    `keras.layers.Embedding`
        Embedding layer, to be used as input to deeper network layers.

    """
#     keyed_vectors = model.wv  # structure holding the result of training
    weights = keyed_vectors.vectors  # vectors themselves, a 2D numpy array    
    index_to_key = keyed_vectors.index_to_key  # which row in `weights` corresponds to which word?

    layer = Embedding(
        input_dim=weights.shape[0],
        output_dim=weights.shape[1],
        weights=[weights],
        trainable=train_embeddings,
    )
    return layer

In [19]:
emb_layer = gensim_to_keras_embedding(wv_from_text)

Создаем модель:

In [20]:
model = Sequential()
model.add(emb_layer)
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Обучение модели:

In [22]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 89s 275ms/step - loss: 0.6377 - accuracy: 0.6033 - val_loss: 0.6205 - val_accuracy: 0.6252
Epoch 2/20
319/319 [==============================] - 88s 276ms/step - loss: 0.6148 - accuracy: 0.6292 - val_loss: 0.6158 - val_accuracy: 0.6305
Epoch 3/20
319/319 [==============================] - 91s 284ms/step - loss: 0.6085 - accuracy: 0.6351 - val_loss: 0.6143 - val_accuracy: 0.6304
Epoch 4/20
319/319 [==============================] - 93s 291ms/step - loss: 0.6035 - accuracy: 0.6417 - val_loss: 0.6137 - val_accuracy: 0.6238
Epoch 5/20
319/319 [==============================] - 85s 266ms/step - loss: 0.5978 - accuracy: 0.6469 - val_loss: 0.6136 - val_accuracy: 0.6312
Epoch 6/20
319/319 [==============================] - 95s 297ms/step - loss: 0.5929 - accuracy: 0.6524 - val_loss: 0.6239 - val_accuracy: 0.6180


In [23]:
score2 = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)

45/45 [==============================] - 6s 131ms/step - loss: 0.6303 - accuracy: 0.6167


Итоговые результаты:

In [28]:
print('Not pretrain Embedding:')
print('Test score:', score1[0])
print('Test accuracy:', score1[1])
print("-"*25)
print('Pretrain Embedding:')
print('Test score:', score2[0])
print('Test accuracy:', score2[1])

Not pretrain Embedding:
Test score: 0.617521345615387
Test accuracy: 0.6253141164779663
-------------------------
Pretrain Embedding:
Test score: 0.63033127784729
Test accuracy: 0.6167173385620117


**Вывод:** по общему score впереди с незначительным отрывом лидирует модель с претренированнным эмбедингом, но по точности стандартная модель чуть лучше. В итоге, по результатам видно, что качество обеих моделей с незначительной разницей примерно одинаково.

---